In [ ]:
with open(path + 'bert_results_ruspell_ru.txt', 'r', encoding='utf-8') as f:
  roberta = f.readlines()

2008

In [ ]:
!pip install transformers sentencepiece torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes accelerate wordpiece peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.4 MB/s eta 0:00:00
  Created wheel for wordpiece: filename=wordpiece-0.0.0-py3-none-any.whl size=1748 sha256=e46aa8c1c9643c6f7d9e13c94af23ed5b28f8b18cd54d5cc92858f4507003901
  Stored in directory: /root/.cache/pip/wheels/a7/07/52/27946dee94359e93d98b96e72cd87d8761b7dc54fae797ef6d
Successfully built wordpiece


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, T5ForConditionalGeneration, AutoModelForSeq2SeqLM #for fb model

In [ ]:
'''
BERT
https://huggingface.co/ai-forever/ruBert-large
###
https://huggingface.co/facebook/nllb-200-3.3B
type T5:
https://huggingface.co/ai-forever/RuM2M100-1.2B
https://huggingface.co/ai-forever/RuM2M100-418M
https://huggingface.co/ai-forever/FRED-T5-large-spell
https://huggingface.co/ai-forever/T5-large-spell
type GPT (en):
https://huggingface.co/mistralai/Mistral-7B-v0.1
https://huggingface.co/IlyaGusev/mythyras_d38_m2
https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora
'''
#TODO: find multilingual datasets for spell-checking tasks
'''
https://github.com/snukky/wikiedits
https://www.cl.cam.ac.uk/research/nl/bea2019st/#data
https://huggingface.co/datasets/wi_locness/viewer/wi?row=2 #+++
'''
#Compute accuracies by each class separately (8 accuracies)
#Compute accuracy on en dataset on punctuation task (silero models)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path = 'drive/MyDrive/Colab Notebooks/DeepLearningModels/Practice №1/'

In [ ]:
corpus = pd.read_csv(path + 'prepared_rlc.csv')
corpus

,Оригинальное предложение,Исправленное предложение,Тег,Ошибка,Исправление
0,Кроме того содержание свинца в выращиваемых пр...,Кроме того содержание свинца в выращиваемых пр...,Infl,свинцем,свинцом
1,Самый главный проект направлен на улучшение эк...,Самый главный проект направлен на улучшение эк...,Infl,свинцем,свинцом
2,Студенты так же борятся за спасение озера Байк...,Студенты также борются за спасение озера Байк...,Infl,борятся,борются
3,Но тот орегонец который ходит на работу после ...,Но тот орегонец который ходит на работу после ...,Infl,школи,школы
4,Я работаю уборчицей няняй поваром и пастором овец,Я работаю уборщицей няняй няней и пастухом,Infl,няняй,няней
...,...,...,...,...,...
3566,Казахский язык богатый своими словарными ...,Казахский богат своим словарным запасом как и ...,Brev,богатый,богат
3567,На первое есть два блюда каппеллетти ин ...,На первое есть два блюда каппеллетти в вине –...,Brev,вкусно,вкусное
3568,Дорогой Джон как ты вчера в интернете я...,Дорогой Джон как ты вчера в интернете я увидел...,Brev,хороши,хороших
3569,Вовторых Вуз в России очень известные,Вовторых Вузы в России очень известны,Brev,известные,известны


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ai-forever/spellcheck_benchmark", 'RUSpellRU')

df_test = pd.DataFrame(dataset['test'])
df_test

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2008 [00:00<?, ? examples/s]

,source,correction,domain
0,﻿есть у вас оформленый и подписаный мною заказ,﻿есть у вас оформленный и подписанный мною заказ,RUSpellRU
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...,RUSpellRU
2,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...,RUSpellRU
3,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...,RUSpellRU
4,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...,RUSpellRU
...,...,...,...
2003,спасибо вам огромное за нормальную новость о е...,спасибо вам огромное за нормальную новость о е...,RUSpellRU
2004,более захватывающее и наглядное обучение возмо...,более захватывающее и наглядное обучение возмо...,RUSpellRU
2005,и вобщем-то все понятно на фоне слухов застави...,и в общем-то все понятно на фоне слухов застав...,RUSpellRU
2006,но всему есть придел,но всему есть предел,RUSpellRU


In [ ]:
original_texts = list(corpus['Оригинальное предложение'])
corrected_texts = list(corpus['Исправленное предложение'])
errors = list(corpus['Ошибка'])
corrections = list(corpus['Исправление'])

In [ ]:
original_texts = dataset['test']['source']
corrections = dataset['test']['correction']

In [ ]:
model_names = [
    'facebook/mbart-large-50-many-to-many-mmt',
    'ai-forever/RuM2M100-1.2B',
    'ai-forever/RuM2M100-418M',
    'ai-forever/FRED-T5-large-spell',
    'ai-forever/T5-large-spell'
]
t5_model_names = [
    'IlyaGusev/fred_t5_ru_turbo_alpaca'
]
gpt_model_names = [
    'IlyaGusev/saiga_7b_lora'
]

In [ ]:
!pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 17.5 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
import regex as re
from nltk import regexp_tokenize, sent_tokenize
import stanza
import string
import copy
import sys

# ---------------- SETTINGS SECTION ----------------

# list of languages to load
languages = ['ru']
attempts_change_ending = 1000

# --------------- LOAD MODELS ----------------

nlp_token = {}
nlp_morph = {}

# set method: Stanza, Trankit, Spacy, Natasha, NLTK
method = {
    'en': 'Stanza',
    'ru': 'Stanza',
}

if 'ru' in languages:
    if method['ru'] == 'Stanza':
        stanza.download('ru')
        nlp_morph['ru'] = stanza.Pipeline(lang="ru", package='syntagrus', processors="tokenize,pos,lemma",
                                        tokenize_pretokenized=True)
        nlp_token['ru'] = stanza.Pipeline(lang="ru", package='gsd', processors='tokenize')

def output_sentences_stanza(text, lang):
    sentences = []
    doc = nlp_token[lang](text)
    for sentence in doc.sentences:
        sentences.append(sentence.text)
    return sentences

def start_position(sentence, text):
    symbols = 1
    try:
        while len(text.split(sentence[:symbols])) > 2 and symbols < len(sentence):
            symbols += 1
    except:
        while len(text.split()) > 2 and symbols < len(sentence):
            symbols += 1
    return text.find(sentence[:symbols])


def correct_sentences_ru(sentences, positions, text):
    sentences_corrected = []
    pos_corrected = []

    for i in range(len(sentences)):
        if len(sentences_corrected) > 0:
            last_sent = sentences_corrected[-1]
            last_pos = pos_corrected[-1]
            pos_ending_symbol = last_pos + len(last_sent)
            # Correct of incorrect splitting "Газета.Ru"
            if sentences[i].startswith('Ru\"'):
                if text[last_pos:last_pos + len(last_sent) + len(sentences[i])] == last_sent + sentences[i]:
                    sentences_corrected[-1] += sentences[i]
                elif text[last_pos:last_pos + len(last_sent) + len(sentences[i]) + 1] == last_sent + text[pos_ending_symbol] + sentences[i]:
                    sentences_corrected[-1] += text[pos_ending_symbol] + sentences[i]
            # Merge sentences if the previous sentence ends with a character from the set ['\"', ',']
            elif len(last_sent) > 1 and last_sent[-1] in ['\"', ','] and \
                    last_sent[-2] not in string.punctuation and text[pos_ending_symbol] != '\n' and \
                    text[last_pos:last_pos + len(last_sent) + len(sentences[i]) + 1] == last_sent + text[pos_ending_symbol] + sentences[i]:
                sentences_corrected[-1] += text[pos_ending_symbol] + sentences[i]
            else:
                sentences_corrected.append(sentences[i])
                pos_corrected.append(positions[i])
        else:
            sentences_corrected.append(sentences[i])
            pos_corrected.append(positions[i])

    return sentences_corrected, pos_corrected


def end_abbr(last_sent, cur_sent, lang):
    # Regular expressions for
    # Г.М.Р., U.S., A.B., А., д.т.н.
    abbr_patterns = {
        'ru': [r'(?:[ ]?\p{Lu}\.){2,3}$', r' [А-Я]\.$', r' (?:[ ]?\p{Ll}\.){3}$'],
        'en': [r'(?:[ ]?\p{Lu}\.){2,3}$', r' [A-Z]\.$', r' (?:[ ]?\p{Ll}\.){3}$'],
        'es': [r'(?:[ ]?\p{Lu}\.){2,3}$'],
        'fr': [r'(?:[ ]?\p{Lu}\.){2,3}$'],
        'de': [r'(?:[ ]?\p{Lu}\.){2,3}$'],
        'uk': [r'(?:[ ]?\p{Lu}\.){2,3}$'],
        'tr': [r'(?:[ ]?\p{Lu}\.){2,3}$'],
        'ar': [r'(?:[ ]?\p{Lu}\.){2,3}$']
    }

    abbrs = {
        # ул., пл., пр., ст., пер., мкр., пгт., дер., пос., им., кв.,
        # г.о., и.о., н.п., т.е., т.к., т.н.
        'ru': [r' [\"(]?([Уу]л|[Пп](л|р|ер|гт|ос)|[Сс]т|[Дд]ер|[Мм]кр|[Вв]л)\.$',
               r' (им|кв|кор|ост|реж|моб|стр|г-н|Дж|St|Vol)\.$',
               r' [ги]\.[ ]?о\.$', r' [нр]\.[ ]?п\.$', r' т\.[ ]?[екн]\.$'],
        'en': [],
        'es': [],
        'fr': [],
        'de': [],
        'uk': [],
        'tr': [],
        'ar': []
    }

    starts = {
        'ru': [r'^(— Ред\.\)|— Авт\.\)|— Прим\. ред\.\)|— РБК\)|— НСН\))'],
        'en': [],
        'es': [],
        'fr': [],
        'de': [],
        'uk': [],
        'tr': [],
        'ar': []
    }

    exeptions = {
        # т.д., т.п., и пр., г.г., г.р., п.п., л.с., в.д., м.р., м.о., мм.рт.ст.
        'ru': [r' т\.[ ]?[дп]\.$', r'и пр\.$', r' г\.[ ]?[гр]\.$', r' п\.[ ]?п\.$', r' л\.[ ]?с\.$',
               r' в\.[ ]?д\.$', r' м\.[ ]?[ро]\.$', r' мм\.[ ]?рт\.[ ]?ст\.$'],
        'en': [],
        'es': [],
        'fr': [],
        'de': [],
        'uk': [],
        'tr': [],
        'ar': []
    }

    find_abbr_pattern = False
    for pattern in abbr_patterns[lang]:
        if len(re.findall(pattern, last_sent)) > 0:
            find_abbr_pattern = True
            break

    find_abbr = False
    if not find_abbr_pattern:
        for pattern in abbrs[lang]:
            if len(re.findall(pattern, last_sent)) > 0:
                find_abbr = True
                break

    find_start = False
    if not find_abbr_pattern and not find_abbr:
        for pattern in starts[lang]:
            if len(re.findall(pattern, cur_sent)) > 0:
                find_start = True
                break

    find_exeption = False
    for pattern in exeptions[lang]:
        if len(re.findall(pattern, last_sent)) > 0:
            find_exeption = True
            break

    if not find_exeption:
        if find_abbr_pattern or find_abbr or find_start:
            return True
        elif lang == 'ru':
            last_sent_splitted = last_sent.split()
            if len(last_sent_splitted) > 1:
                # Sentence ends with 'г.', 'д.', 'п.', 'с.', 'о.'
                if last_sent_splitted[-1] in ['г.', 'д.', 'п.', 'с.' , 'о.'] and not any([ch.isdigit() for ch in last_sent_splitted[-2]]):
                    return True
                # Sentence ends with 'ч.'
                elif last_sent_splitted[-1] == 'ч.' and ' ст.' in last_sent + ' ' + cur_sent:
                    return True
                # Sentence ends with 'р.' and not ends with '10 р.' or '24 тыс. р.'
                elif last_sent_splitted[-1] == 'р.':
                    if not any([ch.isdigit() for ch in last_sent_splitted[-2]]):
                        if len(last_sent_splitted) == 2 or not any([ch.isdigit() for ch in last_sent_splitted[-3]]):
                            return True
                # Sentence ends with 'шт.'
                elif last_sent_splitted[-1] == 'шт.' and cur_sent.startswith('('):
                    return True

    return False


def dot_pattern(last_sent, cur_sent):
    tail = last_sent.split()[-1]
    head = cur_sent.split()[0]
    finds = re.match(r'\"\pL+\.\pL+\"', tail + head)
    if finds is not None and finds.group(0) == tail + head:
        return True
    else:
        return False


def correct_sentences_all_langs(sentences, positions, text, lang):
    sentences_corrected = []
    pos_corrected = []

    for i in range(len(sentences)):
        if len(sentences_corrected) > 0:
            last_sent = sentences_corrected[-1]
            last_pos = pos_corrected[-1]
            # Merge sentences if the previous sentence consists of one words
            if len(regexp_tokenize(last_sent, r'\w+')) == 1 and last_sent[-1] not in string.punctuation and \
                    text[last_pos:last_pos + len(last_sent) + len(sentences[i]) + 1] == last_sent + ' ' + sentences[i]:
                sentences_corrected[-1] += ' ' + sentences[i]
            # Merge sentences
            # if the previous sentence ends with an abbreviations (end_abbr) or
            # if the tail of the previous sentence and head of the next sentence form the pattern "Яндекс.Маркет" (dot_pattern)
            elif len(last_sent) > 1 and (end_abbr(last_sent, sentences[i], lang) or
                                         dot_pattern(last_sent, sentences[i])):
                if text[last_pos:last_pos + len(last_sent) + len(sentences[i]) + 1] == last_sent + ' ' + sentences[i]:
                    sentences_corrected[-1] += ' ' + sentences[i]
                elif text[last_pos:last_pos + len(last_sent) + len(sentences[i])] == last_sent + sentences[i]:
                    sentences_corrected[-1] += sentences[i]
                else:
                    sentences_corrected.append(sentences[i])
                    pos_corrected.append(positions[i])
            else:
                sentences_corrected.append(sentences[i])
                pos_corrected.append(positions[i])
        else:
            sentences_corrected.append(sentences[i])
            pos_corrected.append(positions[i])

    return sentences_corrected, pos_corrected

def preprocess(text, lang, min_sent_len, min_sent_words):
    sentences = []
    if text and lang is not None:
        text_splitted = []
        if method[lang] == 'Stanza':
            text_splitted = output_sentences_stanza(text, lang)
        for sentence in text_splitted:
            if sentence.strip() != '':
                sentences.append(sentence.strip())

    sentences_temp = []
    for sentence in sentences:
        sentences_temp.extend(sentence.split('\n'))
    sentences = sentences_temp

    positions = []
    pos = 0
    for sentence in sentences:
        pos += start_position(sentence, text[pos:])
        positions.append(pos)
        pos += len(sentence)

    sentences, positions = correct_sentences_all_langs(sentences, positions, text, lang)

    if lang == "ru":
        sentences, positions = correct_sentences_ru(sentences, positions, text)

    sentences_filtered = []
    positions_filtered = []
    for sentence, start in zip(sentences, positions):
        words = regexp_tokenize(sentence, r'\w+')
        words_len = sum([len(word) for word in words])
        # Filtering encoding error and short sentences
        if words_len >= len(sentence.replace(' ', '')) - words_len and words_len >= min_sent_len and \
                len(words) >= min_sent_words and any([len(word) > 1 for word in words]):
            sentences_filtered.append(sentence)
            positions_filtered.append(start)

    return sentences_filtered, positions_filtered


def get_sentences(text, lang, min_sent_len=3, min_sent_words=3):
    sentences_dict = []
    sentences, positions = preprocess(text, lang, min_sent_len, min_sent_words)
    for sentence, start in zip(sentences, positions):
        sentences_dict.append({'text': sentence, 'start': start})

    return sentences_dict

def get_tokens_stanza(doc):
    tokens_list = []
    for sentence in doc.sentences:
        for token in sentence.tokens:
            token_splitted = re.split('\"', token.text)
            if len(token_splitted) > 1 and len(token.text) > 1:
                # split tokens, e.g. "Интерфакс\""
                token_splitted = token.text.replace('\"', ' \" ').split()
                for item in token_splitted:
                    if len(item) > 0:
                        tokens_list.append([item, '', '', '', -1, -1])
            else:
                tokens_list.append([token.text, '', '', '', -1, -1])
    return tokens_list

def get_morph_stanza(sentence, message_id=None):
    morph_list = []
    for i in range(len(sentence.words)):
        word = sentence.words[i]
        word_feats = ''
        if word.upos == 'VERB' and word.feats is not None:
            j = 0
            feats = word.feats.split('|')
            while j < len(feats) and not feats[j].startswith('Tense'):
                j += 1
            if j < len(feats):
                word_feats = feats[j]
        elif word.upos == 'NOUN' and word.feats is not None:
            feats = word.feats.split('|')
            j = 0
            while j < len(feats) and not feats[j].startswith('Number'):
                j += 1
            if j < len(feats):
                word_feats = feats[j]
            j = 0
            while j < len(feats) and not feats[j].startswith('Animacy'):
                j += 1
            if j < len(feats):
                if len(word_feats):
                    word_feats += '|'
                word_feats += feats[j]

        if word.lemma is not None:
            morph_list.append([word.text, word.lemma.lower(), word.upos, word_feats, -1, -1])
        else:
            morph_list.append([word.text, word.text.lower(), word.upos, word_feats, -1, -1])
            t = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
            print('%s Text ID: %s, Warning: lemma of %s is undefined' % (t, message_id, word.text))
    return morph_list

def get_word(p, param):
    if len(param) == 3:
        word = p.inflect({param[0], param[1], param[2]})
    else:
        word = p.inflect({param[0], param[1]})
    if word is not None:
        return word.word
    else:
        return None


def get_inflected_words(keyword):
    morph = pymorphy2.MorphAnalyzer()
    cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
    numbers = ['sing', 'plur']
    genders = ['masc', 'femn', 'neut']
    tenses = ['past', 'pres', 'futr']
    p = morph.parse(keyword)[0]
    inflected_words = []
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for param in itertools.product(numbers, tenses, genders):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
        for param in itertools.product(numbers, tenses):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
        for param in itertools.product(tenses, genders):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
    else:
        for param in itertools.product(numbers, cases, genders):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
        for param in itertools.product(numbers, cases):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
        for param in itertools.product(cases, genders):
            word = get_word(p, param)
            if word is not None:
                inflected_words.append(word)
    return list(set(inflected_words))

def correct_keywords_col_bert(text, keywords_col, tokenizer_bert, model_bert, model_name, lang):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    sentences = [sentence for sentence in get_sentences(text, lang, min_sent_len=3, min_sent_words=3)]
    predicted_keywords = []
    print('\t\tSentences:')
    for sentence in sentences:
        print('\t\t\t' + sentence['text'])
    index_sentence = 0
    index_keyword_col = 0
    start_index_search_kw = 0
    corrected_sentence = text
    while index_sentence < len(sentences) and index_keyword_col < len(keywords_col):
        keyword_col = keywords_col[index_keyword_col]
        sentence = sentences[index_sentence]['text']
        if keyword_col in sentence[start_index_search_kw:]:
            print('\n\t\tSentence: ' + sentence)
            start_index_keyword_col = sentence.index(keyword_col, start_index_search_kw)
            prefix = sentence[:start_index_keyword_col].strip()
            tail = sentence[start_index_keyword_col + len(keyword_col):]
            start_index_search_kw += len(keyword_col)
            doc = nlp_morph[lang](keyword_col)
            for i in range(len(sentences)):
              try:
                sentences[i]['grammar'] = get_morph_stanza(doc.sentences[i])
              except:
                sentences[i]['grammar'] = get_morph_stanza(doc.sentences[0])
            grammar = sentences[index_sentence]['grammar']
            keyword_col_token_pos = [[item[0], item[2]] for item in grammar]

            first_noun_index = 0
            while first_noun_index < len(keyword_col_token_pos) - 1 and keyword_col_token_pos[first_noun_index][1] != 'NOUN':
                first_noun_index += 1

            for j in reversed(range(first_noun_index + 1, len(keyword_col_token_pos))):
                tail = ' ' + keyword_col_token_pos[j][0] + tail
            for i in reversed(range(first_noun_index + 1)):
                keyword = keyword_col_token_pos[i][0]
                keyword_stem_nltk = porter_stem.stem(keyword)
                #keyword_stem = correct_stem(keyword_stem, tokenizer_bert)
                bert_predicted_indexes = tokenizer_bert.encode(keyword)[1:-1]
                token_list = []
                for index in bert_predicted_indexes:
                    token_list.append(tokenizer_bert.decode(index))
                if len(token_list) > 1:
                    last_token = token_list[-1].replace('##', '')
                    if len(token_list[-1]) > 1:
                        keyword_stem = ''.join(token_list[:-1])
                    elif len(token_list[-1]) == 1:
                        keyword_stem = ''
                elif len(token_list) == 1:
                    keyword_stem = ''
                else:
                    keyword_stem = None
                if '##' in keyword_stem:
                    keyword_stem = keyword_stem.replace('##', '')
                context = prefix + ''.join(
                    [' ' + x[0] for x in keyword_col_token_pos[:i]]) + ' ' + keyword_stem + tokenizer_bert.mask_token + tail

                print('\t\tContext: ' + context)

                indexed_tokens = tokenizer_bert.encode(context)
                tokens_tensor = torch.tensor([indexed_tokens])
                tokens_tensor = tokens_tensor.to(device)

                mask_token_index = torch.where(tokens_tensor == tokenizer_bert.mask_token_id)[1]

                with torch.no_grad():
                    outputs = model_bert(tokens_tensor)

                del tokens_tensor
                torch.cuda.empty_cache()

                logits = outputs.logits.to(device).cpu().detach()
                masked_token_logits = logits[0, mask_token_index.to(device).cpu().detach(), :]
                index_logit = [(k, masked_token_logits[0, k]) for k in range(len(masked_token_logits[0]))]
                index_logit.sort(key=lambda x: x[1], reverse=True)

                print('\t\tKeyword stem: ' + keyword_stem_nltk)
                k = 0
                found = False
                final_predicted_keyword = ''
                inflected_words = get_inflected_words(keyword)
                while k < attempts_change_ending:
                    predicted_token = tokenizer_bert.decode([index_logit[k][0]])
                    if predicted_token.startswith('##'):
                        predicted_token = predicted_token.replace('##', '')
                    punct = string.punctuation + '...'
                    if predicted_token not in punct and '[' not in predicted_token:
                        if keyword_stem == '':
                            predicted_keyword = predicted_token
                        else:
                            predicted_keyword = keyword_stem + predicted_token
                        predicted_keywords.append(predicted_keyword)
                        print('\t\t\t' + str(k + 1) + ' predicted token: ' + predicted_token +
                          ', predicted keyword: ' + predicted_keyword)
                    else:
                        predicted_keyword = ''

                    if len(predicted_keyword) > 0 and not found and len(predicted_keyword.split()) == 1:
                        if len(inflected_words) > 0:
                            print(inflected_words)
                            morph = pymorphy2.MorphAnalyzer()
                            pos = morph.parse(predicted_keyword)[0].tag.POS
                            pos_compare = False
                            for word in inflected_words:
                                if pos == morph.parse(word)[0].tag.POS:
                                    pos_compare = True
                            if pos_compare and predicted_keyword.lower() in inflected_words:
                                final_predicted_keyword = predicted_keyword
                                found = True
                                break
                            else:
                                if not found and predicted_keyword.lower().startswith(keyword_stem_nltk):
                                    final_predicted_keyword = predicted_keyword
                                    found = True
                                    break
                                if not found and predicted_keyword.lower().startswith(token_list[0]):
                                    final_predicted_keyword = predicted_keyword
                                    found = True
                                    break
                        else:
                            if not found and predicted_keyword.lower().startswith(keyword_stem_nltk):
                                final_predicted_keyword = predicted_keyword
                                found = True
                                break
                            if not found and predicted_keyword.lower().startswith(token_list[0]):
                                final_predicted_keyword = predicted_keyword
                                found = True
                                break
                    k += 1
                if not found and keyword not in tail:
                    tail = ' ' + keyword + tail
                elif found:
                    tail = ' ' + final_predicted_keyword + tail

            print('\t\tCorrected sentence: ' + prefix + tail)
            #sentences[index_sentence] = prefix + tail
            #sentences[index_sentence] = sentences[index_sentence].strip()
            corrected_sentence = prefix + tail
            index_keyword_col += 1
        else:
            start_index_search_kw = 0
            index_sentence += 1
    return corrected_sentence, predicted_keywords

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


In [ ]:
from transformers import GenerationConfig, MBartForConditionalGeneration, MBart50TokenizerFast, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to(device)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

def get_translated_texts(model_name, original_texts):
    outputs_list = []
    model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="ru", tgt_lang="ru").to(device)
    for i, text in enumerate(original_texts):
        print(f'Processing {i} text / {len(original_texts)}')
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id("ru"))
        outputs_list.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    return outputs_list

def get_corrected_texts(model_name, original_texts, model_type='M2M100'):
    outputs_list = []
    if model_type == 'M2M100':
        model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="ru", tgt_lang="ru")
    elif model_type == 'T5':
        model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token="</s>")
    elif model_type =='S2S':
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        generation_config = GenerationConfig.from_pretrained(model_name)
    elif model_type == 'Causal':
        model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        generation_config = GenerationConfig.from_pretrained(model_name)
    elif model_type == 'BART':
        model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
        tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
    elif model_type == 'GPT':
        config = PeftConfig.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
          config.base_model_name_or_path,
          load_in_8bit=True,
          torch_dtype=torch.float16,
          device_map="auto"
        )
        model = PeftModel.from_pretrained(
          model,
          model_name,
          torch_dtype=torch.float16,
          device_map="auto"
        )
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        generation_config = GenerationConfig.from_pretrained(model_name)
    else:
        raise ValueError('Incorrect model name')
    for i, text in enumerate(original_texts):
        print(f'Processing {i} text / {len(original_texts)}')
        if model_type == 'T5':
            prefix = 'Исправь: '
            length = len(text)
            text = prefix + text
            encodings = tokenizer([text], return_tensors="pt").to(device)
            generated_tokens = model.generate(
                **encodings, eos_token_id=tokenizer.eos_token_id)
        elif model_type == 'M2M100':
            encodings = tokenizer([text], return_tensors="pt").to(device)
            generated_tokens = model.generate(
                **encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
        elif model_type == 'S2S':
            prefix = 'Исправь грамматические ошибки и верни исправленное предложение: '
            text = prefix + text
            encodings = tokenizer([text], return_tensors="pt").to(device)
            generated_tokens = model.generate(
                **encodings, generation_config=generation_config
                )
        elif model_type == 'GPT':
            DEFAULT_SYSTEM_PROMPT = "<s>system\nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s>"
            prompt = '<s>user\nИсправь грамматические ошибки и верни исправленное предложение: '
            output = generate(model, tokenizer, prompt + text + '</s><s>bot\n', generation_config)
        elif model_type == 'BART':
            tokenizer.src_lang = "ru_RU"
            encodings = tokenizer([text], return_tensors="pt").to(device)
            generated_tokens = model.generate(
              **encodings,
              forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
            )
        else:
            raise ValueError('Incorrect model name')
        if model_type == 'T5':
          outputs_list.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0][:length])
        elif model_type == 'M2M100':
          outputs_list.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
        elif model_type == 'S2S':
          outputs_list.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
        elif model_type == 'BART':
          outputs_list.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])
        elif model_type == 'GPT':
          outputs_list.append(output)
          with open(path + 'gpt_results.txt', 'a', encoding='utf-8') as f:
            f.write(output + '\n')
          print(output)
          torch.cuda.empty_cache()
    return outputs_list

In [ ]:
facebook_outputs = get_translated_texts(model_names[0], original_texts)
corpus[model_names[0]] = facebook_outputs

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
sber_list = []
for model_name in gpt_model_names:
    sber_outputs = get_corrected_texts(model_name, original_texts, model_type='GPT')
    corpus[model_name] = sber_outputs

ValueError: ignored

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=90029e55cc7aad474aa8f04ab1a1b781110fa60719f2228020cafa0a45b01c56
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import pymorphy2
import string
import torch
import copy
import requests
import itertools

import torch.nn.functional as F
from nltk.stem.snowball import SnowballStemmer

In [ ]:
bert_model_names = [
    'ai-forever/ruRoberta-large',
    'ai-forever/ruBert-large'
]

In [ ]:
def extract_words(text):
    text = text.strip()
    sents = text.split()
    words = []
    for word in sents:
        word = word.lower().replace('ё', 'е')
        i = len(word) - 1
        while i >= 0 and not (word[i].isalpha() or word[i].isdigit()):
            i -= 1
        if i < 0:
            continue
        word = word[:(i+1)]
        while len(word) > 0 and not (word[0].isalpha() or word[0].isdigit()):
            word = word[1:]
        if word != "" and len(word) > 2:
            words.append(word)
    return words

In [ ]:
total_corrections = []
for i in range(len(df_test)):
  corrections = extract_words(df_test.iloc[i]['correction'])
  sources = extract_words(df_test.iloc[i]['source'])
  total_corrections.append(list(set(sources).difference(corrections)))

In [ ]:
for model_name in bert_model_names[1:]:
  #bert_outputs = []
  tokenizer_bert = AutoTokenizer.from_pretrained(model_name)
  model_bert = AutoModelForMaskedLM.from_pretrained(model_name)
  model_bert.eval()
  model_bert.to(device)
  porter_stem = SnowballStemmer("russian")
  for i in range(len(original_texts)):
    print(f'Processing {i} text / {len(original_texts)}')
    keywords_col = total_corrections[i]
    lang = 'ru'
    corrected_sentence, keywords = correct_keywords_col_bert(original_texts[i], keywords_col, tokenizer_bert, model_bert, model_name, lang)
    with open(path + 'bert_results_ruspell_ru.txt', 'a', encoding='utf-8') as f:
      f.writelines(corrected_sentence + '\n')
    #bert_outputs.append(corrected_sentence)
  #corpus[model_name] = bert_outputs

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of the model checkpoint at ai-forever/ruBert-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Выходные данные были обрезаны до нескольких последних строк (5000).
['конвеера', 'конвеерами', 'конвеер', 'конвееру', 'конвеере', 'конвеером', 'конвеерам', 'конвееров', 'конвеерах']
		Corrected sentence: в храме кормят всех желающих процесс поставлен на конвеер люди приходят в любое время дня
Processing 1010 text / 2008
		Sentences:
			в первые гды перестройки было такое мнение дескать мы все привезем изза границы

		Sentence: в первые гды перестройки было такое мнение дескать мы все привезем изза границы
		Context: в первые г[MASK] перестройки было такое мнение дескать мы все привезем изза границы
		Keyword stem: гды
			3 predicted token: д, predicted keyword: гд
		Corrected sentence: в первые гд перестройки было такое мнение дескать мы все привезем изза границы

		Sentence: в первые гды перестройки было такое мнение дескать мы все привезем изза границы
		Context: в первые гды перестройки было такое мнение дескать мы все привезем из[MASK] границы
		Keyword stem: изз
			1 predicted tok

In [ ]:
len(bert_outputs)

NameError: ignored

In [ ]:
with open(path + 'bert_results1.txt', 'w', encoding='utf-8') as f:
  bert_outputs = f.readlines()

NameError: ignored

In [ ]:
for i, text in enumerate(bert_outputs):
    bert_outputs[i] += '\n'
with open(path + 'bert_results1.txt', 'w', encoding='utf-8') as f:
  f.writelines(bert_outputs)

In [ ]:
corpus.to_csv(path + 'bert_full_corpus.csv')